In [1]:
# This program is based on the Udemy course "Python for Finance: Investment Fundamentals & Data Analytics" 
# by Martin and Ned from 365
# Link to the course: https://www.udemy.com/course/python-for-finance-investment-fundamentals-data-analytics/
# This is a jupyter notebook file, better to run in jupyter notebook enviorment.
# Author: Martin and Ned, 365 Team. 
# Updated by: Kuo L. 2022-01-28

import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# use any 3 tickers, I just use 3 vanguard ETFs.
# you can put more than 3 tickers, just note that the more tickers in, 
# the more calculation needed to plot efficient frontier. change num_simulation to a higher number as needed.
num_simulation = 1000
assets = ['VOO', 'VYM', 'VHT']
pf_data = pd.DataFrame()

for a in assets:
    #you can chagne the start date to any date in 'yyyy-mm-dd' format.
    pf_data[a] = wb.DataReader(a, data_source = 'yahoo', start = '2018-1-1')['Adj Close'] 

In [ ]:
pf_data.head() #makre sure companies were public at starting date

In [ ]:
pf_data.tail() #check companies still open and public at ending date

In [ ]:
(pf_data / pf_data.iloc[0] * 100).plot(figsize=(10, 5)) #Plot of company performance without considering dividends

In [ ]:
log_returns = np.log(pf_data / pf_data.shift(1))

In [ ]:
log_returns.mean() * 250

In [ ]:
log_returns.cov() * 250

In [ ]:
log_returns.corr()

### Expected Portfolio Return:

np.sum(weights * log_returns.mean()) * 250

### Expected Portfolio Variance:

np.dot(weights.T, np.dot(log_returns.cov() * 250, weights))

### Expected Portfolio Volatility:

np.sqrt(np.dot(weights.T,np.dot(log_returns.cov() * 250, weights)))

***

In [ ]:
num_assets = len(assets)

pfolio_returns = []
pfolio_volatilities = []
w0 = []
w1 = []
w2 = []

for x in range (num_simulation):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    w0.append(weights[0])
    w1.append(weights[1])
    w2.append(weights[2])
    pfolio_returns.append(np.sum(weights * log_returns.mean()) * 250)
    pfolio_volatilities.append(np.sqrt(np.dot(weights.T,np.dot(log_returns.cov() * 250, weights))))
    
pfolio_returns = np.array(pfolio_returns)
pfolio_volatilities = np.array(pfolio_volatilities)
w0 = np.array(w0)
w1 = np.array(w1)
w2 = np.array(w2)

pfolio_returns, pfolio_volatilities, w0, w1, w2

In [ ]:
portfolios = pd.DataFrame({'Return': pfolio_returns, 'Volatility': pfolio_volatilities, assets[0]: w0, assets[1]: w1, assets[2]: w2})

In [ ]:
portfolios.head()

In [ ]:
portfolios.tail()

In [ ]:
portfolios.sort_values(by=['Volatility'])

In [ ]:
portfolios.plot(x='Volatility', y='Return', kind='scatter', figsize=(10, 6));
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')

In [ ]:
#generate path and name then export
path = 'results/'
name = '_'.join(assets) + '.csv'
path_name = path + name 
portfolios.sort_values(by=['Volatility']).to_csv(path_name)